In [16]:
%logstop
%logstart -rtq ~/.logs/ml.py append
import seaborn as sns
sns.set()

In [3]:
from static_grader import grader
from sklearn import set_config
set_config(display="diagram")

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [4]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
--2021-07-19 16:04:04--  http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.195.65
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.195.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3398120 (3.2M) [text/csv]
Saving to: ‘./ml-data/providers-train.csv’

     0K .......... .......... .......... .......... ..........  1% 3.23M 1s
    50K .......... .......... .......... .......... ..........  3% 6.70M 1s
   100K .......... .......... .......... .......... ..........  4%  404K 3s
   150K .......... .......... .......... .......... ..........  6% 79.6M 2s
   200K .......... .......... .......... .......... ..........  7% 6.82M 2s
   250K .......... .......... .......... .......... ..........  9% 60.3M 2s
   300K .......... .......... .......... .......... .....

We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [5]:
import numpy as np
import pandas as pd

#silents warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` method of your estimator, not a DataFrame. This means that your model must work with both data types. You can handle this by adding a test (and optional conversion) in the `predict` method of your custom class, similar to the `ColumnSelectTransformer` given below in Question 2.  

In [8]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}

    def fit(self, X, y):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.group_averages['global_average'] = y.mean()
        return self

    def predict(self, X):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages.get(group, self.group_averages['global_average']) for group in X[self.grouper]]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [9]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [10]:
state_model.predict(data.sample(5))

[8054.977611940299,
 3490.756838905775,
 13476.666666666666,
 32754.970588235294,
 19835.32283464567]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [11]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [12]:
grader.score.ml__state_model(state_model.predict)

Your score: 1.000


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** As mentioned above in Question 1, when the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [13]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer())
])

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [14]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {'estimator__n_estimators': range(20, 100, 10),
              'estimator__max_depth': range(3, 15, 1)}

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('estimator', RandomForestClassifier(n_estimators=60, max_depth = 5))
])

gs = RandomizedSearchCV(simple_features_model, param_distributions = param_grid, cv = 5, n_iter = 50, n_jobs = 4, verbose = 1)

gs.fit(data, fine_counts > 0)

gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   34.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:  3.0min finished


{'estimator__n_estimators': 30, 'estimator__max_depth': 4}

In [17]:
simple_features_model = Pipeline([
    ('simple', simple_features),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
])

simple_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imputer', SimpleImputer())])),
                ('estimator',
                 RandomForestClassifier(min_samples_leaf=100,
                                        n_estimators=200))])

In [18]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score: 1.033


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [24]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder (categories = 'auto', sparse = False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [25]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the categorical_features, fit it to the data, and submit its predict_proba method to the grader.

In [26]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
])

In [27]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('owner',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))])),
                                                ('cert',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                 ('ohe',
                                                                  OneHotEncoder(sparse=False))]))])),
                ('estimator',
                 RandomForestClassifier(min_samples_leaf=100,
                                        n_estimators=200))])

In [28]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score: 1.003


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [29]:
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

In [30]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [31]:
business_model = Pipeline([
    ('features', business_features),
    ('ploy', PolynomialFeatures(3)),
    ('estimator', RandomForestClassifier(n_estimators=200, min_samples_leaf = 100))
    
])

In [32]:
business_model.fit(data, fine_counts > 0);

In [33]:
grader.score.ml__business_model(positive_probability(business_model))

Your score: 1.001


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [34]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X,pd.DataFrame):
               X = pd.DataFrame(X)
        return (pd.to_datetime( X[self.t2_col])
                -pd.to_datetime( X[self.t1_col])).values.reshape(-1,1).astype(float)

In [35]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [36]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    ("poly", PolynomialFeatures(2)),
    ("svd", TruncatedSVD(20) ),
    ("classifier", RandomForestRegressor(n_estimators=200, min_samples_leaf = 100))
])

In [38]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                                                                                                                   'CCRC_FACIL',
                                                                                                                                   'SFF',
                                                                                                                                   'CHOW_LAST_12MOS',
                                                                                                                                   'SPRINKLER_STATUS',
                                                                                                                                   'EXP_TOTAL',
                                                                                                                                   'ADJ_TOTAL'])),
                                                                                                 ('imputer',
                                                                                                  SimpleImputer())])),
                                                                                ('categorical',
                                                                                 FeatureUnion(transformer_...
                                                                                  'CYCLE_1_NFROMDEFS',
                                                                                  'CYCLE_1_NFROMCOMP',
                                                                                  'CYCLE_1_DEFS_SCORE',
                                                                                  'CYCLE_1_NUMREVIS',
                                                                                  'CYCLE_1_REVISIT_SCORE',
                                                                                  'CYCLE_1_TOTAL_SCORE'])),
                                                ('time',
                                                 TimedeltaTransformer(t1_col='CYCLE_1_SURVEY_DATE',
                                                                      t2_col='CYCLE_2_SURVEY_DATE'))])),
                ('poly', PolynomialFeatures()),
                ('svd', TruncatedSVD(n_components=20)),
                ('classifier',
                 RandomForestRegressor(min_samples_leaf=100,
                                       n_estimators=200))])

In [39]:
grader.score.ml__survey_model(survey_model.predict)

Your score: 1.230


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*